In [35]:
import re
import numpy as np
from sklearn.model_selection import train_test_split


In [55]:
f_shakespeare = open('../../data/shakespeare_cleaned.txt','r')
shakespeare = f_shakespeare.read()

f_obama = open('../../data/obama_cleaned.txt','r')
obama = f_obama.read()



In [46]:
n = 70 #amount of words in each data point
obama_words = obama.split(' ')
c_obama = [" ".join(obama_words[i:i+n]) for i in range(0, len(obama_words), n)]

print(np.average([len(x) for x in c_obama]))
print(len(c_obama))


384.4099099099099
444


In [59]:
n = 68 #amount of words/sentences in each data point
shake_words = shakespeare.split(' ')

#split based on n sentences - disabled for now
#c_shakespeare = [". ".join(shake_words[i:i+n]) for i in range(0, len(shake_words), n)]

#split based on n words
c_shakespeare = [" ".join(shake_words[i:i+n]) for i in range(0, len(shake_words), n)]

np.random.seed(59374)
np.random.shuffle(c_shakespeare)

print(np.average([len(x) for x in c_shakespeare]))
print(len(c_shakespeare))


370.10420193935664
12994


In [64]:
print(len(c_shakespeare[:444]))
print(len(c_obama))

l_shakespeare = [(x, "shakespeare") for x in c_shakespeare]
l_obama = [(x, "obama") for x in c_obama]

total_data = c_shakespeare[:444] + c_obama
np.random.seed(59374)
np.random.shuffle(total_data)

train, test = train_test_split(total_data)

classifier = 

444
444
666
222
